In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv


# API KEY 정보로드
load_dotenv()
# gemini_api_key = os.getenv("GEMINI_API_KEY")
# gemini_api_key

In [ ]:
import os


def seraph_langsmith(project_name=None, set_enable=True):

    if set_enable:
        langchain_key = os.environ.get("LANGCHAIN_API_KEY", "")
        langsmith_key = os.environ.get("LANGSMITH_API_KEY", "")

        # 더 긴 API 키 선택
        if len(langchain_key.strip()) >= len(langsmith_key.strip()):
            result = langchain_key
        else:
            result = langsmith_key

        if result.strip() == "":
            print(
                "LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954"
            )
            return

        os.environ["LANGSMITH_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGSMITH_TRACING"] = "true"  # true: 활성화
        os.environ["LANGSMITH_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name=}")
        print("LANGSMITH_PROJECT:", os.environ.get("LANGSMITH_PROJECT"))

    else:
        os.environ["LANGSMITH_TRACING"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("02-Gemini-LLM")
# seraph_langsmith("02-Gemini-LLM")

In [ ]:
def print_llm_response(llm, response) -> None:
    """
    LLM 응답을 가독성 좋게 출력하는 함수

    Parameters:
        llm: LLM 객체 (예: ChatGoogleGenerativeAI)
        response: LLM 응답 객체 (invoke 결과)
    """
    model_name = getattr(llm, "model", "Unknown Model")
    print(f"\n📌 [Model: {model_name}] Reply\n{'─' * 60}")
    if hasattr(response, "content") and isinstance(response.content, str):
        print(response.content)
    else:
        print(f"[⚠️ 경고] 예상치 못한 응답 타입: {type(response)}\n{response}")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import SecretStr
import os


# 키 로드
api_key = os.getenv("GEMINI_API_KEY")

# LLM 생성
# 모델 종류: https://ai.google.dev/gemini-api/docs/models?hl=ko
# gemini-2.5-pro
# gemini-2.5-flash
# gemini-2.5-flash-lite
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=SecretStr(str(api_key)),
    temperature=0.5,
)

# 질문
question = "대한민국의 수도는 어디인가요?"

# 응답 생성
response = llm.invoke(question)
print_llm_response(llm, response)

In [ ]:
from langchain_teddynote.messages import stream_response

# 스트림 방식으로 질의
# answer 에 스트리밍 답변의 결과를 받습니다.
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")
stream_response(answer)

## 프롬프트 캐싱

- 참고 링크: https://platform.openai.com/docs/guides/prompt-caching

프롬프트 캐싱 기능을 활용하면 반복하여 동일하게 입력으로 들어가는 토큰에 대한 비용을 아낄 수 있습니다.

다만, 캐싱에 활용할 토큰은 고정된 PREFIX 를 주는 것이 권장됩니다.

아래의 예시에서는 `<PROMPT_CACHING>` 부분에 고정된 토큰을 주어 캐싱을 활용하는 방법을 설명합니다.

In [ ]:
from langchain_teddynote.messages import stream_response

very_long_prompt = """
당신은 매우 친절한 AI 어시스턴트 입니다. 
당신의 임무는 주어진 질문에 대해 친절하게 답변하는 것입니다.
아래는 사용자의 질문에 답변할 때 참고할 수 있는 정보입니다.
주어진 정보를 참고하여 답변해 주세요.

<WANT_TO_CACHE_HERE>
#참고:
**Prompt Caching**
Model prompts often contain repetitive content, like system prompts and common instructions. OpenAI routes API requests to servers that recently processed the same prompt, making it cheaper and faster than processing a prompt from scratch. This can reduce latency by up to 80% and cost by 50% for long prompts. Prompt Caching works automatically on all your API requests (no code changes required) and has no additional fees associated with it.

Prompt Caching is enabled for the following models:

gpt-4.1 (excludes gpt-4.1-2024-05-13 and chatgpt-4.1-latest)
gpt-4.1-mini
o1-preview
o1-mini
This guide describes how prompt caching works in detail, so that you can optimize your prompts for lower latency and cost.

Structuring prompts
Cache hits are only possible for exact prefix matches within a prompt. To realize caching benefits, place static content like instructions and examples at the beginning of your prompt, and put variable content, such as user-specific information, at the end. This also applies to images and tools, which must be identical between requests.

How it works
Caching is enabled automatically for prompts that are 1024 tokens or longer. When you make an API request, the following steps occur:

Cache Lookup: The system checks if the initial portion (prefix) of your prompt is stored in the cache.
Cache Hit: If a matching prefix is found, the system uses the cached result. This significantly decreases latency and reduces costs.
Cache Miss: If no matching prefix is found, the system processes your full prompt. After processing, the prefix of your prompt is cached for future requests.
Cached prefixes generally remain active for 5 to 10 minutes of inactivity. However, during off-peak periods, caches may persist for up to one hour.

Requirements
Caching is available for prompts containing 1024 tokens or more, with cache hits occurring in increments of 128 tokens. Therefore, the number of cached tokens in a request will always fall within the following sequence: 1024, 1152, 1280, 1408, and so on, depending on the prompt's length.

All requests, including those with fewer than 1024 tokens, will display a cached_tokens field of the usage.prompt_tokens_details chat completions object indicating how many of the prompt tokens were a cache hit. For requests under 1024 tokens, cached_tokens will be zero.

What can be cached
Messages: The complete messages array, encompassing system, user, and assistant interactions.
Images: Images included in user messages, either as links or as base64-encoded data, as well as multiple images can be sent. Ensure the detail parameter is set identically, as it impacts image tokenization.
Tool use: Both the messages array and the list of available tools can be cached, contributing to the minimum 1024 token requirement.
Structured outputs: The structured output schema serves as a prefix to the system message and can be cached.
Best practices
Structure prompts with static or repeated content at the beginning and dynamic content at the end.
Monitor metrics such as cache hit rates, latency, and the percentage of tokens cached to optimize your prompt and caching strategy.
To increase cache hits, use longer prompts and make API requests during off-peak hours, as cache evictions are more frequent during peak times.
Prompts that haven't been used recently are automatically removed from the cache. To minimize evictions, maintain a consistent stream of requests with the same prompt prefix.
Frequently asked questions
How is data privacy maintained for caches?

Prompt caches are not shared between organizations. Only members of the same organization can access caches of identical prompts.

Does Prompt Caching affect output token generation or the final response of the API?

Prompt Caching does not influence the generation of output tokens or the final response provided by the API. Regardless of whether caching is used, the output generated will be identical. This is because only the prompt itself is cached, while the actual response is computed anew each time based on the cached prompt. 

Is there a way to manually clear the cache?

Manual cache clearing is not currently available. Prompts that have not been encountered recently are automatically cleared from the cache. Typical cache evictions occur after 5-10 minutes of inactivity, though sometimes lasting up to a maximum of one hour during off-peak periods.

Will I be expected to pay extra for writing to Prompt Caching?

No. Caching happens automatically, with no explicit action needed or extra cost paid to use the caching feature.

Do cached prompts contribute to TPM rate limits?

Yes, as caching does not affect rate limits.

Is discounting for Prompt Caching available on Scale Tier and the Batch API?

Discounting for Prompt Caching is not available on the Batch API but is available on Scale Tier. With Scale Tier, any tokens that are spilled over to the shared API will also be eligible for caching.

Does Prompt Caching work on Zero Data Retention requests?

Yes, Prompt Caching is compliant with existing Zero Data Retention policies.
</WANT_TO_CACHE_HERE>

#Question:
{}

"""

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

prompt = "프롬프트 캐싱 기능에 대해 2문장으로 설명하세요"

answer = llm.invoke(prompt)

print(f"[답변]: {answer.content}")

# Groq에서는 token 사용량 정보가 거의 제공되지 않음
# response_metadata 자체는 존재하지만, 내부에 token_usage 관련 필드가 비어 있을 수 있음
print(f"[메타정보]: {answer.response_metadata}")

## 멀티모달 모델(이미지 인식)

멀티모달은 여러 가지 형태의 정보(모달)를 통합하여 처리하는 기술이나 접근 방식을 의미합니다. 이는 다음과 같은 다양한 데이터 유형을 포함할 수 있습니다.

- 텍스트: 문서, 책, 웹 페이지 등의 글자로 된 정보
- 이미지: 사진, 그래픽, 그림 등 시각적 정보
- 오디오: 음성, 음악, 소리 효과 등의 청각적 정보
- 비디오: 동영상 클립, 실시간 스트리밍 등 시각적 및 청각적 정보의 결합

> ❗ 전제: Groq는 멀티모달(MM) 입력을 지원하지 않습니다

In [ ]:
from langchain_groq import ChatGroq

# Groq 모델 생성
llm = ChatGroq(
    temperature=0.1,
    model="llama3-70b-8192",
)

# 이미지 입력은 지원 안되므로, 텍스트만으로 질문 구성
question = (
    "이 이미지는 다음 URL에 있습니다:\n"
    "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg\n"
    "이미지를 볼 수 없다고 가정하고, 사람들이 어떤 느낌을 받을지 상상해서 설명해줘."
)

# 응답 생성
response = llm.invoke(question)
print(f"[답변]: {response.content}")

## System, User 프롬프트 수정

In [ ]:
from langchain_groq import ChatGroq

# Groq LLM 객체
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.1,
)

# 시스템 프롬프트와 유저 프롬프트를 통합
prompt = f"""
당신은 표(재무제표)를 해석하는 금융 AI 어시스턴트입니다.
당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다.

[표]
회사의 재무제표가 여기에 있다고 가정하세요.

요청: 재무제표에서 흥미로운 사실을 정리해서 알려주세요.
"""

# 응답 생성
response = llm.invoke(prompt)
print(f"[답변]: {response.content}")